In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [14]:
EDU0_Raw = "../wonder/data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_0-.1).txt"
EDU1_Raw = "data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_.1-.2).txt"
EDU2_Raw = "data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_.2-.3).txt"
EDU3_Raw = "data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_.3-.4).txt"
EDU4_Raw = "data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_.4-.5).txt"
EDU5_Raw = "data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_.5-.6).txt"
EDU6_Raw = "data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_.6-.7).txt"
EDU7_Raw = "data_edu/Multiple Cause of Death, 2018-2021, Single Race (EDU_.7-.8).txt"

In [17]:
EDU0 = pd.read_csv(EDU0_Raw, sep='\t', skipfooter=1, engine='python')
EDU1 = pd.read_csv(EDU1_Raw, sep='\t', skipfooter=228, engine='python')
EDU2 = pd.read_csv(EDU2_Raw, sep='\t', skipfooter=138, engine='python')
EDU3 = pd.read_csv(EDU3_Raw, sep='\t', skipfooter=128, engine='python')
EDU4 = pd.read_csv(EDU4_Raw, sep='\t', skipfooter=105, engine='python')
EDU5 = pd.read_csv(EDU5_Raw, sep='\t', skipfooter=138, engine='python')
EDU6 = pd.read_csv(EDU6_Raw, sep='\t', skipfooter=128, engine='python')
EDU7 = pd.read_csv(EDU7_Raw, sep='\t', skipfooter=105, engine='python')

In [18]:
EDU0

,Notes,Census Region,Census Region Code,Year,Year Code,Gender,Gender Code,Single Race 6,Single Race 6 Code,Five-Year Age Groups,Five-Year Age Groups Code,Deaths,Population,Crude Rate
0,NaN,Census Region 3: South,CENS-R3,2018.0,2018.0,Female,F,White,2106-3,35-39 years,35-39,12.0,22465.0,Unreliable
1,NaN,Census Region 3: South,CENS-R3,2018.0,2018.0,Male,M,White,2106-3,30-34 years,30-34,17.0,25885.0,Unreliable
2,NaN,Census Region 3: South,CENS-R3,2018.0,2018.0,Male,M,White,2106-3,35-39 years,35-39,15.0,25762.0,Unreliable
3,NaN,Census Region 3: South,CENS-R3,2018.0,2018.0,Male,M,White,2106-3,45-49 years,45-49,12.0,26508.0,Unreliable
4,NaN,Census Region 3: South,CENS-R3,2018.0,2018.0,Male,M,White,2106-3,50-54 years,50-54,10.0,26048.0,Unreliable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,http://wonder.cdc.gov/wonder/help/mcd-expanded...,None,None,NaN,NaN,None,None,None,None,None,None,NaN,NaN,None
89,7. The population figures used in the calculat...,None,None,NaN,NaN,None,None,None,None,None,None,NaN,NaN,None
90,resident population that is under one year of ...,None,None,NaN,NaN,None,None,None,None,None,None,NaN,NaN,None
91,Group.,None,None,NaN,NaN,None,None,None,None,None,None,NaN,NaN,None
